# Tarea modelos regularizados

1. Cargue el archivo 'california_housing_train', que está en la carpeta 'sample_data' de Colab. **Este va a ser su conjunto de datos de entrenamiento**. Cargue el archivo 'california_housing_train', que está en la carpeta 'sample_data' de Colab. **Este va a ser su conjunto de datos de prueba**.

**Nota: ** Cómo en este caso ya tiene los conjuntos de entrenamiento y prueba, **no necesita hacer partición aleatoria con train_test_split**.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error

In [ ]:
df_train = pd.read_csv('/content/sample_data/california_housing_train.csv')
df_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [ ]:
df_test = pd.read_csv('/content/sample_data/california_housing_test.csv')
df_test.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0


In [ ]:
y_test = df_test['median_house_value']
X_test = df_test.drop('median_house_value', axis=1)

2. Implemente un modelo Ridge que prediga la variable 'median_house_value' a partir de las otras variables.
Sintonice $\lambda$ por validación cruzada usando como métrica RMSE.

Entregue:
- El mejor RMSE obtenido y el $\lambda$ correspondiente.
- El error de validación y el error de prueba.
- Los coeficientes asociados a cada característica del modelo.

In [ ]:
y_train = df_train['median_house_value']
X_train = df_train.drop('median_house_value', axis=1)

model = RidgeCV(alphas=np.logspace(-4,8), scoring='neg_root_mean_squared_error', cv=5)
model.fit(X_train, y_train)
print(f'Menor error de generalizacion: {-model.best_score_:.3f} con lambda: {model.alpha_}')
print(f'Error de prueba: {np.sqrt(mean_squared_error(y_test, model.predict(X_test))):.3f}')

Menor error de generalizacion: 71929.438 con lambda: 719.6856730011514
Error de prueba: 69847.124


In [ ]:
pesos = pd.DataFrame(data=model.coef_, index=X_train.columns, columns=['Coeficiente'])
pesos.loc['intercepto'] = model.intercept_
pesos

,Coeficiente
longitude,-3.717266e+04
latitude,-3.735895e+04
housing_median_age,1.247028e+03
total_rooms,-8.781763e+00
total_bedrooms,1.082764e+02
population,-3.872352e+01
households,5.966085e+01
median_income,4.074491e+04
intercepto,-3.109843e+06


3. Implemente un modelo Lasso que prediga la variable 'median_house_value' a partir de las otras variables.
Sintonice $\lambda$ por validación cruzada usando como métrica RMSE.

Entregue:
- El mejor RMSE obtenido y el $\lambda$ correspondiente.
- El error de validación y el error de prueba.
- Los coeficientes asociados a cada característica del modelo.

In [ ]:
model = LassoCV(alphas=np.logspace(-4,2), cv=5)
model.fit(X_train, y_train)
print(f'El mejor RMSE obtenido: {np.sqrt(mean_squared_error(y_train, model.predict(X_train))):.3f}')
print(f'Error de prueba: {np.sqrt(mean_squared_error(y_test, model.predict(X_test))):.3f}')

El mejor RMSE obtenido: 69459.296
Error de prueba: 69762.948


In [ ]:
pesos = pd.DataFrame(data=model.coef_, index=X_train.columns, columns=['Coeficiente'])
pesos.loc['intercepto'] = model.intercept_
pesos

,Coeficiente
longitude,-4.276221e+04
latitude,-4.257311e+04
housing_median_age,1.156106e+03
total_rooms,-8.409734e+00
total_bedrooms,1.170770e+02
population,-3.850035e+01
households,4.632768e+01
median_income,4.052554e+04
intercepto,-3.588300e+06


4. Implemente un modelo Ridge que prediga la variable 'median_house_value', pero previamente haga selección de características mediante un modelo Lasso. Sintonice $\lambda$ por validación cruzada usando como métrica RMSE.

Entregue:
- El mejor RMSE obtenido y el $\lambda$ correspondiente.
- El error de validación y el error de prueba.
- Los coeficientes asociados a cada característica del modelo.

In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=100000)
lasso.fit(X_train, y_train)
pesos_lasso = pd.DataFrame(data=lasso.coef_, index=X_train.columns, columns=['Coeficiente'])
pesos_lasso

,Coeficiente
longitude,0.000000
latitude,-0.000000
housing_median_age,927.789709
total_rooms,33.756131
total_bedrooms,-207.436842
population,-59.073725
households,234.183531
median_income,6760.939299


Se eliminan longitud y  latitud

In [ ]:
selected_features = pesos_lasso[pesos_lasso['Coeficiente'] != 0].index
selected_features

Index(['housing_median_age', 'total_rooms', 'total_bedrooms', 'population',
       'households', 'median_income'],
      dtype='object')

In [ ]:
X_train_sel = df_train[selected_features]
y_train_sel = df_train['median_house_value']

X_test_sel = df_test[selected_features]
y_test_sel = df_test['median_house_value']

model = RidgeCV(alphas=np.logspace(-8,2), scoring='neg_root_mean_squared_error', cv=5)
model.fit(X_train_sel, y_train_sel)
print(f'Error de generalizacion: {np.sqrt(mean_squared_error(y_train, model.predict(X_train_sel))):.3f} con lambda {model.alpha_}')
print(f'Error de prueba: {np.sqrt(mean_squared_error(y_test, model.predict(X_test_sel))):.3f}')

Error de generalizacion: 75934.076 con lambda 1e-08
Error de prueba: 76216.575


Indica que lo mejor es un modelo sin regularizar, ya que el alpha se pega al valor mas pequeño en el logspace


In [ ]:
pesos = pd.DataFrame(data=model.coef_, index=X_train_sel.columns, columns=['Coeficiente'])
pesos.loc['intercepto'] = model.intercept_
pesos

,Coeficiente
housing_median_age,1880.014083
total_rooms,-19.955921
total_bedrooms,99.524772
population,-35.018235
households,127.225320
median_income,48033.459277
intercepto,-47624.111719
